- Neste notebook faremos a conversão de dados categóricos (respostas para as perguntas), em valores números, atribuindo um _score_ para cada resposta.


- Um problema que possuímos nos dados é que as respostas não são sempre Sim/Não por exemplo. Mas sim, por exemplo, binárias (Sim/Não), númericas (variando de 1 até 10) ou em escala (Sempre, Quase nunca, Nunca).


- O código desse notebook foi baseado no código original presente no [banco de dados do Kaggle](https://www.kaggle.com/ruslankl/lgbt-survey-analysis). Porém, usaremos os valores para outros fins.

In [1]:
import numpy as np
import pandas as pd

### Lendo os dados

In [2]:
df_DailyLife                    = pd.read_csv('cleaned-survey-data-2012/DailyLife.csv')
df_RightsAwareness              = pd.read_csv('cleaned-survey-data-2012/RightsAwareness.csv')
df_Discrimination               = pd.read_csv('cleaned-survey-data-2012/Discrimination.csv')
df_ViolenceAndHarrassment       = pd.read_csv('cleaned-survey-data-2012/ViolenceAndHarrassment.csv')
df_SubsetSize                   = pd.read_csv('cleaned-survey-data-2012/SubsetSize.csv')
df_TransgenderSpecificQuestions = pd.read_csv('cleaned-survey-data-2012/TransgenderSpecificQuestions.csv')

### Calculando a proporção de cada subset

- Usaremos os valores presentes no SubsetSize para calcular a proporção de cada subset por país, considerando que, a proporção geral (primeira linha do arquivo SubsetSize.csv) esteja correta.

In [3]:
# Merge entre SubsetSize com DailyLife e renomeando as colunas
df_SubsetSize = df_SubsetSize.merge(df_DailyLife[['CountryID', 'CountryName']], how='left').drop_duplicates().reset_index(drop=True)
df_SubsetSize.rename(columns={'Lesbian women': 'Lesbian', 'Gay men': 'Gay'}, inplace=True)
df_SubsetSize.replace(np.nan, 'EU Total', inplace=True)

# Calculando a razão de cada subset por país
subsets = ['Lesbian', 'Gay', 'Bisexual women', 'Bisexual men', 'Transgender']
for ss in subsets:
    df_SubsetSize[ss + ' ratio'] = df_SubsetSize[ss] / df_SubsetSize['N']
    df_SubsetSize[ss + ' ratio'] = df_SubsetSize[ss + ' ratio'].apply(lambda x: round(x, 3))

# Exibindo o dataframe até então
df_SubsetSize.head()

,CountryID,N,Lesbian,Gay,Bisexual women,Bisexual men,Transgender,CountryName,Lesbian ratio,Gay ratio,Bisexual women ratio,Bisexual men ratio,Transgender ratio
0,EU Total,93079,15236,57448,6424,7200,6771,EU Total,0.164,0.617,0.069,0.077,0.073
1,AT,2543,437,1558,164,217,167,Austria,0.172,0.613,0.064,0.085,0.066
2,BE,2901,424,1998,132,182,165,Belgium,0.146,0.689,0.046,0.063,0.057
3,BG,1033,226,474,139,88,106,Bulgaria,0.219,0.459,0.135,0.085,0.103
4,CY,265,41,164,18,17,25,Cyprus,0.155,0.619,0.068,0.064,0.094


### Criando um dataframe contendo os pesos para cada país

In [4]:
# Computando o peso para cada país
for ss in subsets:
    df_SubsetSize[ss + ' weight'] = df_SubsetSize[ss + ' ratio'] / df_SubsetSize.loc[df_SubsetSize['CountryName'] == 'EU Total', ss + ' ratio'].values[0]
    df_SubsetSize[ss + ' weight'] = df_SubsetSize[ss + ' weight'].apply(lambda x: round(x, 3))
    
# Criando uma nova tabela contendo apenas as razões e outra contendo apenas os pesos
df_SubsetRatios  = df_SubsetSize.drop(['N'] + subsets + [ss + ' weight' for ss in subsets], axis=1)
df_SubsetWeights = df_SubsetSize.drop(['N'] + subsets + [ss + ' ratio'  for ss in subsets], axis=1)

### Exibindo os novos dataframes

In [5]:
df_SubsetRatios.head()

,CountryID,CountryName,Lesbian ratio,Gay ratio,Bisexual women ratio,Bisexual men ratio,Transgender ratio
0,EU Total,EU Total,0.164,0.617,0.069,0.077,0.073
1,AT,Austria,0.172,0.613,0.064,0.085,0.066
2,BE,Belgium,0.146,0.689,0.046,0.063,0.057
3,BG,Bulgaria,0.219,0.459,0.135,0.085,0.103
4,CY,Cyprus,0.155,0.619,0.068,0.064,0.094


In [6]:
df_SubsetWeights.head()

,CountryID,CountryName,Lesbian weight,Gay weight,Bisexual women weight,Bisexual men weight,Transgender weight
0,EU Total,EU Total,1.000,1.000,1.000,1.000,1.000
1,AT,Austria,1.049,0.994,0.928,1.104,0.904
2,BE,Belgium,0.890,1.117,0.667,0.818,0.781
3,BG,Bulgaria,1.335,0.744,1.957,1.104,1.411
4,CY,Cyprus,0.945,1.003,0.986,0.831,1.288


### Salvandos os novos dataframes

In [7]:
df_SubsetRatios.to_csv('weighted-survey-data-2012/SubsetRatios.csv', index=False)
df_SubsetWeights.to_csv('weighted-survey-data-2012/SubsetWeights.csv', index=False)